# License Plate Object Detection

## Installation for Using NVIDIA GPU Device
License Plate Object Detection is the part of our Deep Learning Pipeline where we need to identify Region of Interest of the license plate that we want to recognize. Our Object Detection model will be using a ***YOLOv5*** method which has been created by ***ultralytics***. All Credits goes to every people who are involve in bringing YOLOv5 Method to live. The code can be accessed using this link https://github.com/ultralytics/yolov5.

The training of the data will be using NVIDIA GeForce GTX 1660 Ti device. But there are some things that we need to prepare for this project such as:
1. Installing CUDA Toolkit version 10.2 (use this [link](https://developer.nvidia.com/cuda-10.2-download-archive) for downloading it)
2. Installing CuDNN version 8.3.0 (or pick other version that is compatible with CUDA Toolkit version, check this [link](https://developer.nvidia.com/rdp/cudnn-archive) for further information)
3. Installing NVIDIA driver from this [link](https://www.nvidia.com/download/index.aspx) and choose the driver based on your GPU device name and type.
4. Installing Visual Studio 2019 using this [link](https://visualstudio.microsoft.com/thank-you-downloading-visual-studio/?sku=community&rel=16&utm_medium=microsoft&utm_source=docs.microsoft.com&utm_campaign=download+from+relnotes&utm_content=vs2019ga+button) 

## Setting Up Environment
Python environment can be created using [anaconda](https://www.anaconda.com/) or [pipenv](https://pipenv.pypa.io/en/latest/) package by Python. In this project, pipenv is a tool that has been chosen for setting up environment. For starting things off, download the any Python version and then run ***pip install pipenv*** for installing pipenv package. Then setting up the environment at your project directory folder by running ***python -m pipenv --python 3.8.6***. Then you want to access or activate the environment using ***python -m pipenv shell***.

Next, we need to install libaries to enable pytorch to access GPU by installing python packages by using this command
***pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html***

This command can be run in the jupyter notebook or in the command line (***make sure to run the command in the python environment we just created***)

In [1]:
# # run this command if it takes too long just run it on the command prompt, inside the python environment
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
import shutil
import torch
import os
from IPython.display import Image, clear_output

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

C:\Users\USER\.virtualenvs\license-plate-object-detection-uHB0SpUB\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete. Using torch 1.9.0+cu102 (NVIDIA GeForce GTX 1660 Ti)


## Clone Repository
Clone yolov5 repository by ***ultralytics*** from this link https://github.com/ultralytics/yolov5

***Make sure that git has already installed in your computer and enabled to be executed from any file directory*** (set git into the environment variables)

In [3]:
# clone the repo from this link 'https://github.com/ultralytics/yolov5.git'
git_https = 'https://github.com/ultralytics/yolov5.git'
folder_name = 'yolov5'
if not os.path.exists(os.path.join(os.getcwd(), folder_name)):
    !git clone {git_https}
else:
    !cd yolov5 && git pull
    print(folder_name, 'already exists and up to date')

# install requirements of yolov5
!cd yolov5 && pip install -r requirements.txt
# install roboflow for data pulling
!pip install roboflow

Updating 614ef11..9a7f289
Fast-forward
 .github/workflows/docker.yml                     | 54 ++++++++++++++++++++++++
 .github/workflows/greetings.yml                  |  2 +-
 README.md                                        |  2 +-
 models/tf.py                                     |  3 +-
 utils/docker/Dockerfile                          |  2 +-
 utils/docker/{Dockerfile-M1 => Dockerfile-arm64} |  5 ++-
 utils/docker/Dockerfile-cpu                      |  4 +-
 7 files changed, 64 insertions(+), 8 deletions(-)
 create mode 100644 .github/workflows/docker.yml
 rename utils/docker/{Dockerfile-M1 => Dockerfile-arm64} (90%)
yolov5 already exists and up to date


From https://github.com/ultralytics/yolov5
   614ef11..9a7f289  master     -> origin/master


  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Attempting uninstall: certifi
    Found existing installation: certifi 2021.10.8
    Uninstalling certifi-2021.10.8:
      Successfully uninstalled certifi-2021.10.8
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.3
    Uninstalling idna-3.3:
      Successfully uninstalled idna-3.3


## Dataset Preparation
After collecting our data, we will be using roboflow (can be accessed from this [link](https://roboflow.com/)) which is a tool for anotating region of interest. In this case, the region of interest is license plate. After anotating, we can export into any form of a dataset that will be accepted by our model. We can export it manually or using an API. 

In [4]:
# pulling costum-data created from roboflow website using API
####################################TEMPLATE EXAMPLE#########################################
# from roboflow import Roboflow
# rf = Roboflow(api_key="***************")
# project = rf.workspace("augmented-startups").project("vehicle-registration-plates-trudk")
# dataset = project.version(2).download("yolov5")

In [5]:
# Setting up location for the dataset
DATASET_PARENT_FOLDER = os.path.join(os.getcwd(), 'datasets')
DATASET_FOLDER_NAME = 'Car-License-Plate-1' # filled later becase the dataset is still being collected
DATASET_LOCATION = os.path.join(DATASET_PARENT_FOLDER, DATASET_FOLDER_NAME)

if not os.path.exists(DATASET_LOCATION):
    print("dataset doesn't exists")
else:
    print("dataset already exists")
    

print('location of dataset = ', DATASET_LOCATION)

dataset already exists
location of dataset =  C:\Users\USER\Documents\GitHub\VePay-Go-ML\license-plate-object-detection\datasets\Car-License-Plate-1


Checking up the data.yaml inside the **DATASET_LOCATION** because the object detection API that is used needs it to find information where we put our data.

In [6]:
# take a look inside the data.yaml file
import yaml
with open(os.path.join(DATASET_LOCATION, "data.yaml"), "r") as stream:
    try:
        content = yaml.safe_load(stream) 
        # print the content of data.yaml file
        # we need to change the train and val path
        for key, vals in content.items():
            print(key, '=', vals)
            
        num_classes= content['nc']
        print('num of classes = ', num_classes)
    except yaml.YAMLError as exc:
        print(exc)

# number of classes
print(num_classes)

path = ../datasets/Car-License-Plate-1
train = train/images
val = valid/images
nc = 1
names = ['Plate']
num of classes =  1
1


## Configure Yolov5 Model
We can do this configuring the .yaml file of that model that has been provided inside the yolov5 repo that we clone

In [7]:
import os

model_directory = os.path.join(os.getcwd(), 'yolov5', 'models')
print('\n' + 30*'#'+ ' Yolo V5 default version ' + 30*'#')
for file in os.listdir(model_directory):
    if file.endswith('.yaml'):
        print(file)

print('\n' + 30*'#'+ ' Yolo V5 version 6 ' + 30*'#')
for file in os.listdir(model_directory+'/hub'):
    if file in ['yolov5l6.yaml',  'yolov5m6.yaml', 'yolov5n6.yaml', 'yolov5s6.yaml', 'yolov5x6.yaml']:
        print(file)


############################## Yolo V5 default version ##############################
custom_yolov5l6.yaml
custom_yolov5m6.yaml
custom_yolov5n6.yaml
custom_yolov5s6.yaml
custom_yolov5x6.yaml
yolov5l.yaml
yolov5m.yaml
yolov5n.yaml
yolov5s.yaml
yolov5x.yaml

############################## Yolo V5 version 6 ##############################
yolov5l6.yaml
yolov5m6.yaml
yolov5n6.yaml
yolov5s6.yaml
yolov5x6.yaml


For this project we will be using the Yolo V5 version 6 as our pretrained model for transfer learning the size of the model will be picked based capability of local machine. Don't use heavy model for training if teh hardware capabilty can't keep up with it. There must be a trade off between using **-small size and accuracy model-** or **-big size and high accuracy model-**. 

In [8]:
yolov5_v6_yamls = [
    'yolov5l6.yaml',  
    'yolov5m6.yaml', 
    'yolov5n6.yaml', 
    'yolov5s6.yaml', 
    'yolov5x6.yaml',
]

for file in os.listdir(os.path.join(model_directory, 'hub')):
    if file in yolov5_v6_yamls:
        with open(os.path.join(model_directory, 'hub', file), "r") as stream:
            try:
                content = yaml.safe_load(stream)
                print(file)
                print(content, '\n')
            except yaml.YAMLError as exc:
                print(exc)

yolov5l6.yaml
{'nc': 80, 'depth_multiple': 1.0, 'width_multiple': 1.0, 'anchors': [[19, 27, 44, 40, 38, 94], [96, 68, 86, 152, 180, 137], [140, 301, 303, 264, 238, 542], [436, 615, 739, 380, 925, 792]], 'backbone': [[-1, 1, 'Conv', [64, 6, 2, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 3, 'C3', [128]], [-1, 1, 'Conv', [256, 3, 2]], [-1, 6, 'C3', [256]], [-1, 1, 'Conv', [512, 3, 2]], [-1, 9, 'C3', [512]], [-1, 1, 'Conv', [768, 3, 2]], [-1, 3, 'C3', [768]], [-1, 1, 'Conv', [1024, 3, 2]], [-1, 3, 'C3', [1024]], [-1, 1, 'SPPF', [1024, 5]]], 'head': [[-1, 1, 'Conv', [768, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 8], 1, 'Concat', [1]], [-1, 3, 'C3', [768, False]], [-1, 1, 'Conv', [512, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 6], 1, 'Concat', [1]], [-1, 3, 'C3', [512, False]], [-1, 1, 'Conv', [256, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 4], 1, 'Concat', [1]], [-1, 3, 'C3', [256, False]], [-1, 1, 'Conv', [256, 3, 2]], [[-1, 20], 1, 

We can just change the ***nc*** into any number according to number of class that we want to predict. In this case, we use ***1*** because we only want to predict one class only which is license plate. Create a file call custom model .yaml by copying it.

In [9]:
# create a new costume .yaml folder to enable the model to predict 1 class only
# copy the contents of the .yaml file which is identified by the model's name
model_folder = os.path.join(os.getcwd(), 'yolov5', 'models', 'hub')

for file in os.listdir(model_directory+'/hub'):
    if file in yolov5_v6_yamls:
        yaml_ori_path = os.path.join(model_directory, 'hub', file)
        custom_yaml_name = 'custom_'+file
        custom_yaml_path = os.path.join(model_directory, custom_yaml_name)
        
        if not os.path.exists(custom_yaml_path):
            shutil.copy(yaml_ori_path, custom_yaml_path)
            print(yaml_ori_path, 'has been copied to ', custom_yaml_path)
        else:
            print(custom_yaml_name, 'already exists')

custom_yolov5l6.yaml already exists
custom_yolov5m6.yaml already exists
custom_yolov5n6.yaml already exists
custom_yolov5s6.yaml already exists
custom_yolov5x6.yaml already exists


Edit the .yaml file manually by using a notepad or using other code editor. The result can be seen below

In [10]:
# results after editing the .yaml files for model configuration
custom_yolov5_v6_yamls = [
    'custom_yolov5l6.yaml',  
    'custom_yolov5m6.yaml', 
    'custom_yolov5n6.yaml', 
    'custom_yolov5s6.yaml', 
    'custom_yolov5x6.yaml',
]

for file in os.listdir(model_directory):
    if file in custom_yolov5_v6_yamls:
        with open(os.path.join(model_directory, file), "r") as stream:
            try:
                content = yaml.safe_load(stream)
                print(file)
                print(content, '\n')
            except yaml.YAMLError as exc:
                print(exc)

custom_yolov5l6.yaml
{'nc': 1, 'depth_multiple': 1.0, 'width_multiple': 1.0, 'anchors': [[19, 27, 44, 40, 38, 94], [96, 68, 86, 152, 180, 137], [140, 301, 303, 264, 238, 542], [436, 615, 739, 380, 925, 792]], 'backbone': [[-1, 1, 'Conv', [64, 6, 2, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 3, 'C3', [128]], [-1, 1, 'Conv', [256, 3, 2]], [-1, 6, 'C3', [256]], [-1, 1, 'Conv', [512, 3, 2]], [-1, 9, 'C3', [512]], [-1, 1, 'Conv', [768, 3, 2]], [-1, 3, 'C3', [768]], [-1, 1, 'Conv', [1024, 3, 2]], [-1, 3, 'C3', [1024]], [-1, 1, 'SPPF', [1024, 5]]], 'head': [[-1, 1, 'Conv', [768, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 8], 1, 'Concat', [1]], [-1, 3, 'C3', [768, False]], [-1, 1, 'Conv', [512, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 6], 1, 'Concat', [1]], [-1, 3, 'C3', [512, False]], [-1, 1, 'Conv', [256, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 4], 1, 'Concat', [1]], [-1, 3, 'C3', [256, False]], [-1, 1, 'Conv', [256, 3, 2]], [[-1, 20

## Training Yolov5 Model
Since we will be preparing a python script (from /yolov5/train.py) for training, there are some arguments that we need to consider which are:
1. ***img:*** define input image size
2. ***batch:*** determine batch size
3. ***epochs:*** define the number of training epochs. (Note: often, 3000+ are common here!).
4. ***data:*** set the path to our yaml file.
5. ***cfg:*** specify our model configuration.
6. ***weights:*** specify a custom path to weights (if the file doesn't exist the model pretrained weights will be downloaded automatically)
7. ***cache:*** cache images for faster training. The default value for this argument is using 'RAM' we can change it to using 'disk' for more storage.
8. ***project:*** folder for results
9. ***name:*** result names inside the **project** folder
10. ***freeze:*** for input how many layers that want to be freezed start from index zero to n-1 layer. For transfer learning we usually freeze the **backbone layers**.
11. ***hyp:*** hyperparameter for training

### Hyperparameter Tuning

### Training Script Generation

In [17]:
img = 640
batch = 32
epochs = 5
data = os.path.join(DATASET_LOCATION, 'data.yaml')
cfg = os.path.join(model_directory, 'custom_yolov5s6.yaml')
weights = 'yolov5s6.pt'
workers= 3
cache = 'disk'
project = 'train_results' 
name = 'train'
freeze = ''
for layer in range(12):
    freeze += str(layer)+ ' '

command = "cd yolov5 && python train.py --img {} --batch {} --epochs {} --data {} --cfg {} --weights {} --workers {} --cache {} --project {} --name {} --freeze {}".\
          format(img, batch, epochs, data, cfg, weights, workers, cache, project, name, freeze)
print(command)

cd yolov5 && python train.py --img 640 --batch 32 --epochs 5 --data C:\Users\USER\Documents\GitHub\VePay-Go-ML\license-plate-object-detection\datasets\Car-License-Plate-1\data.yaml --cfg C:\Users\USER\Documents\GitHub\VePay-Go-ML\license-plate-object-detection\yolov5\models\custom_yolov5s6.yaml --weights yolov5s6.pt --workers 3 --cache disk --project train_results --name train --freeze 0 1 2 3 4 5 6 7 8 9 10 11 


## Evaluating Model
This step is of evaluation we will check its precision, recall, etc. Using tools from weight and biases it will be a lot easier, because they provide visualization at a whole different level. Confidence threshold can be configured and we can see it realtime in the dashboard after training process

In [2]:
# making sure that the wandb package has already installed
# !pip install wandb

Run the above command from the command prompt (**inside the python environment that has been created**) so that we can see the training logs.

## Validating Yolov5 Model
The reason we need to validate the model is just to be sure that our model perform as expected. The difference between validation that is being run by this script and the training script are lies within the argument of Intersection Over Union (IOU) threshold and confidence threshold. These two arguments determine how our model will perform by selecting object abouve the ***confidence threshold*** and selecting bounding box which has an IOU above ***IOU threshold***. there are some arguments that we need to consider which are:
1. ***img:*** define input image size
2. ***data:*** set the path to our yaml file.
3. ***weights:*** specify a custom path from our trained model weights
4. ***conf-thres:*** threshold to determine which object to choose that has confidence value above it
5. ***iou-thres:*** threshold to determine which bounding box to choose that has IOU value above it
6. ***verbose:*** report mean average precision by class

<img src="images/iou.jpeg">

*source: https://medium.com/analytics-vidhya/iou-intersection-over-union-705a39e7acef*

In [20]:
img = 640
data = os.path.join(DATASET_LOCATION, 'data.yaml')
weights = 'C:/Users/USER/Documents/GitHub/VePay-Go-ML/license-plate-object-detection/yolov5/train_results/train6/weights/best.pt'
conf_thres = 0.5
iou_thres = 0.5

command = "cd yolov5 && python val.py --img {} --data {} --weights {} --conf-thres {} --iou-thres {} --verbose".\
          format(img, data, weights, conf_thres, iou_thres)
print(command)

cd yolov5 && python val.py --img 640 --data C:\Users\USER\Documents\GitHub\VePay-Go-ML\license-plate-object-detection\datasets\Car-License-Plate-1\data.yaml --weights C:/Users/USER/Documents/GitHub/VePay-Go-ML/license-plate-object-detection/yolov5/train_results/train6/weights/best.pt --conf-thres 0.5 --iou-thres 0.5 --verbose


## Inferencing using Yolov5 Model
Since we will be preparing a python script (from /yolov5/detect.py) for testing, there are some arguments that we need to consider which are:
1. ***img:*** define input image size
2. ***source:*** input data
3. ***conf:*** confidence level treshold
4. ***weights:*** specify a custom path from our trained model weights
5. ***name:*** file name of the detect results

In [18]:
# example
img=640
source= 'C:/Users/USER/Documents/GitHub/VePay-Go-ML/license-plate-object-detection/datasets/Car-License-Plate-1/test/images'
conf =0.5
weights = 'C:/Users/USER/Documents/GitHub/VePay-Go-ML/license-plate-object-detection/yolov5/train_results/train6/weights/best.pt'

command = "cd yolov5 && python detect.py --weights {} --img {} --conf {} --source {}".\
          format(weights, img, conf, source)
print(command)

cd yolov5 && python detect.py --weights C:/Users/USER/Documents/GitHub/VePay-Go-ML/license-plate-object-detection/yolov5/train_results/train6/weights/best.pt --img 640 --conf 0.5 --source C:/Users/USER/Documents/GitHub/VePay-Go-ML/license-plate-object-detection/datasets/Car-License-Plate-1/test/images


## Export Model to TensorflowJS
Lastly, we export the model for deployment in Website using Javascript.

In [17]:
# export model
model_weights_path = input('Please input model path = ')
!cd yolov5 && python export.py --weights {model_weights_path} --include tfjs

Please input model path = yolov5s.pt
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(1, 640, 640, 3)]   0           []                               
                                                                                                  
 tf_conv (TFConv)               (1, 320, 320, 32)    3488        ['input_1[0][0]']                
                                                                                                  
 tf_conv_1 (TFConv)             (1, 160, 160, 64)    18496       ['tf_conv[0][0]']                
                                                                                                  
 tfc3 (TFC3)                    (1, 160, 160, 64)    18624       ['tf_conv_1[0][0]']              
                                                         

export: data=C:\Users\USER\Documents\GitHub\VePay-Go-ML\license-plate-object-detection\yolov5\data\coco128.yaml, weights=['yolov5s.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tfjs']
YOLOv5  v6.1-196-g614ef11 Python-3.8.6 torch-1.9.0+cu102 CPU


  0%|          | 0.00/14.1M [00:00<?, ?B/s]
  2%|1         | 272k/14.1M [00:00<00:05, 2.68MB/s]
  4%|3         | 544k/14.1M [00:00<00:06, 2.32MB/s]
  7%|7         | 1.03M/14.1M [00:00<00:04, 2.82MB/s]
 11%|#1        | 1.59M/14.1M [00:00<00:03, 3.78MB/s]
 14%|#3        | 1.98M/14.1M [00:00<00:03, 3.78MB/s]
 17%|#6        | 2.35M/14.1M [00:00<00:03, 3.81MB/s]
 19%|#9        | 2.73M/14.1M [00:00<00:03, 3.81MB/s]
 22%|##2       | 3.11M/14.1M [00:00<00:03, 3.60MB/s]
 25%|##4       | 3.50M/14.1M [00:01<00